In [ ]:
%reset -f
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
import os
import importlib
import vep_prob_models
import lib.plots.stan

In [ ]:
data_dir = 'datasets/id002_cj'
results_dir = 'results/tmp'
os.makedirs(results_dir,exist_ok=True)
os.makedirs(f'{results_dir}/logs',exist_ok=True)
os.makedirs(f'{results_dir}/figures',exist_ok=True)

consts = dict()
obs = dict()
network = np.load(f'{data_dir}/CJ_network.npz')
consts['SC'] = network['SC']
consts['SC'] = consts['SC'] / consts['SC'].max()
consts['SC'][np.diag_indices(consts['SC'].shape[0])] = 0
consts['gain'] = network['gain_mat']

slp = np.load(f'{data_dir}/CJ_fit_trgt.npz')['fit_trgt']
obs['slp'] = slp[0:-1:10]
slp_sqrd = obs['slp']**2
consts['nt'], consts['ns'] = obs['slp'].shape
obs['snsr_pwr'] = slp_sqrd.mean(axis=0)
# obs['snsr_pwr'] = obs['snsr_pwr'] / obs['snsr_pwr'].max()

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(211)
plt.plot(obs['slp'], color='black', alpha=0.3);
plt.xlabel('Time', fontsize=15)
plt.title('SLP - Modelled data')
plt.subplot(212)
plt.bar(np.arange(1,consts['gain'].shape[0]+1), obs['snsr_pwr'], color='black', alpha=0.3)
plt.plot(obs['snsr_pwr'].T, color='black', alpha=0.3)
plt.xlabel('Electrode', fontsize=15)
plt.ylabel('Power', fontsize=15)
plt.savefig(f'{results_dir}/figures/fitting_data.png')

In [ ]:
# importlib.reload(vep_prob_models)

consts['nn'] = consts['SC'].shape[0]
# consts['ns'] = consts['gain'].shape[0]
consts['I1'] = 3.1
consts['time_step'] = 0.1
consts['x_init'] = -2.0*np.ones(consts['nn'])
consts['z_init'] = 3.5*np.ones(consts['nn'])
consts['eps_slp'] = 0.1
consts['eps_snsr_pwr'] = 5.0/150.0
# consts['alpha'] = 0.1

params_init = dict()
# params_init['x0_star_linear__'] = np.zeros(consts['nn'])
# # params_init['x_init_star'] = np.zeros(consts['nn'])
# # params_init['z_init_star'] = np.zeros(consts['nn'])
# params_init['amplitude_star_linear__'] = 0.0
# params_init['offset_star_linear__'] = 0.0
# params_init['K_star_linear__'] = 0.0
# params_init['tau0_star_linear__'] = 0.0
# params_init['eps_star'] = 0.0
# params_init['eps_snsr_pwr_star'] = 0.0
# params_init['eps_slp_star'] = 0.0
# params_init['eps_snsr_pwr_star'] = 0.0
params_init['x0'] = -3.0*np.ones(consts['nn'])
params_init['amplitude'] = 1.0
params_init['offset'] = 0.0
params_init['K'] = 0.0
params_init['tau0'] = 30

# max_depth = 15
# delta = 0.95
# fname_suffix = f'hyper_fixed_md{max_depth}_delta{delta}'

In [ ]:
vep_mdl = vep_prob_models.vep_ode_uniform_priors(consts=consts, obs=obs)

In [ ]:
niters = 5000
optmzr = pm.SVGD(n_particles=5, jitter=0.1, temperature=1, model=vep_mdl.model, start=params_init, random_seed=123)
post_approx = optmzr.fit(niters)
# for i in range(10):
#     optmzr.refine(1000)
# trace = pm.sample(draws=1000, start=params_init, tune=1000, model=vep_mdl.model, chains=2, nuts_kwargs={'max_treedepth':10, 'target_accept':0.8})

In [ ]:
try:
    i = 0
    while(True):
        print(f'Iterations {niters+i*100} - {niters+(i+1)*100}')
        optmzr.refine(10, progressbar=False)
        trace = optmzr.approx.sample(100)
        i += 1
except FloatingPointError as err:
    print(err)

### Save the trace

In [ ]:
save_dir_name = 'samples'
os.makedirs(f'{results_dir}/{save_dir_name}', exist_ok=True)
pm.backends.save_trace(trace,f'{results_dir}/{save_dir_name}', overwrite=True)

In [ ]:
# trace = pm.backends.load_trace(f'{results_dir}', model=vep_mdl.model)
trace = np.load(f'{results_dir}/samples/0/samples.npz')

In [ ]:
with vep_mdl.model:
    trace = post_approx.sample(1000)

In [ ]:
with vep_mdl.model:
    trace = pm.backends.text.load(results_dir)

In [ ]:
syn_data = np.load('datasets/id002_cj/CJ_syn_tvb_ez=6-34_pz=27-5-11.npz')
x0_true = syn_data['x0']
ez = syn_data['ez']
pz= np.concatenate((syn_data['pz_x0'], syn_data['pz_kplng']))
lib.plots.stan.x0_violin_syn(trace['x0'], x0_true, ez, pz, figsize=(25,5))
lib.plots.stan.pair_plots(trace, ['K', 'tau0', 'amplitude', 'offset'])
lib.plots.stan.plot_source(trace['x'].mean(axis=0), trace['z'].mean(axis=0), ez, pz)
slp = trace['mu_slp'].mean(axis=0)
snsr_pwr = trace['mu_snsr_pwr'].mean(axis=0)
lib.plots.stan.plot_fit_target(data_pred={'slp':slp,'snsr_pwr':snsr_pwr}, data_true={'slp':obs['slp'],'snsr_pwr':obs['snsr_pwr']})

In [ ]:
post_pred = pm.sample_posterior_predictive(trace, model=vep_mdl.model)

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(post_pred['slp'][20], 'r', alpha=0.3);
plt.plot(obs['slp'], 'k', alpha=0.3);
plt.figure(figsize=(25,5))
plt.bar(np.r_[0:consts['ns']], obs['snsr_pwr'], color='black', alpha=0.3)
plt.bar(np.r_[0:consts['ns']], trace['mu_snsr_pwr'].mean(axis=0), color='red', alpha=0.5)

In [ ]:
trace = np.load('results/exp10/exp10.66/0/samples.npz')
ep = dyn_model.Epileptor_2D()
params = dict()
params['nn'] = consts['nn']
params['nt'] = consts['nt']
params['time_step'] = consts['time_step']
params['I1'] = consts['I1']
params['SC'] = consts['SC']
params['x_init'] = consts['x_init']
params['z_init'] = consts['z_init']
x = np.zeros((trace['x0'].shape[0], consts['nt'], consts['nn']))
z = np.zeros((trace['x0'].shape[0], consts['nt'], consts['nn']))
slp = np.zeros((trace['x0'].shape[0], consts['nt'], consts['ns']))
for i in range(trace['x0'].shape[0]):
    params['tau0'] = trace['tau0'][i]
    params['K'] = trace['K'][i]
    params['x0'] = trace['x0'][i]
#     params['x_init'] = trace['x_init'][i]
#     params['z_init'] = trace['z_init'][i]
    sim_out = ep.sim(params)
    x[i] = sim_out['x']
    z[i] = sim_out['z']
    slp[i] = 4*(np.log(consts['gain']@np.exp(x[i].T))  - 2.9).T

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(slp.mean(axis=0), 'r', alpha=0.3);
plt.plot(obs['slp'], 'k', alpha=0.3);